In [1]:
import pstats

p = pstats.Stats('output.prof')
p.sort_stats('time').print_stats(10)

Mon Nov 27 14:19:28 2023    output.prof

         190452936 function calls (168192971 primitive calls) in 182.341 seconds

   Ordered by: internal time
   List reduced from 1318 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      257   75.257    0.293   75.257    0.293 {method 'acquire' of '_thread.lock' objects}
     2317   19.427    0.008   70.209    0.030 C:\Users\denis.iglesias\Miniconda3\envs\neo4j\lib\site-packages\ijson\backends\python.py:49(Lexer)
 10589037   11.600    0.000   35.005    0.000 C:\Users\denis.iglesias\Miniconda3\envs\neo4j\lib\site-packages\ijson\backends\python.py:134(parse_value)
  3943957    9.397    0.000   16.628    0.000 c:\Users\denis.iglesias\Documents\GitHub\TSM-AdvDaBa_TP22\tests\fichier_tests.py:95(preprocess_json)
22242398/2317    7.168    0.000   70.297    0.030 {method 'send' of 'generator' objects}
 10589851    6.882    0.000    6.882    0.000 {method 'search' of 're.Pattern' objects}
  58243

In [9]:
import httpx
import orjson
import time

def preprocess_line(line):
    return line.replace('NaN', 'null').replace('NumberInt(', '').replace(')', '')

def preprocess_json(url, batch_size=5000, max_retries=3, timeout=10):
    attempts = 0
    while attempts < max_retries:
        try:
            with httpx.stream('GET', url, timeout=timeout) as response:
                response.raise_for_status()
                batch, json_buffer, in_object = [], '', False
                for chunk in response.iter_bytes():
                    lines = chunk.decode().split('\n')
                    for line in lines:
                        line = preprocess_line(line)
                        if line.startswith('{'):
                            in_object = True  # Start of a new JSON object
                        if in_object:
                            json_buffer += line
                        if (line.endswith('},') or line.endswith('}')) and in_object:
                            in_object = False  # End of the current JSON object
                            print("Detected end of the current JSON object")
                            print("JSON Buffer before parsing:", json_buffer)
                            if json_buffer:
                                try:
                                    json_object = orjson.loads(json_buffer)
                                    print("Parsed JSON Object:", json_object)
                                    batch.append(json_object)
                                    if len(batch) == batch_size:
                                        yield batch
                                        batch = []
                                    json_buffer = ''  # Reset buffer after processing
                                except orjson.JSONDecodeError as e:
                                    print(f"Error parsing JSON: {e} in buffer: {json_buffer}")
                                    json_buffer = ''  # Reset buffer on error
                # Process any remaining JSON object
                if json_buffer:
                    try:
                        yield [orjson.loads(json_buffer)]
                    except orjson.JSONDecodeError as e:
                        print(f"Error parsing JSON: {e} in buffer: {json_buffer}")
            break
        except (httpx.HTTPError, httpx.TimeoutException) as e:
            attempts += 1
            print(f"Attempt {attempts}/{max_retries} failed: {e}. Retrying...")
            time.sleep(1)
        if attempts == max_retries:
            raise httpx.HTTPError(f"Failed to retrieve data after {max_retries} attempts.")

# Example usage
url = "http://vmrum.isc.heia-fr.ch/biggertest.json"
for batch in preprocess_json(url):
    print("Received a batch of JSON objects")
    print(batch)  # Or your processing logic

Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
Detected end of the current JSON object
